# Fan In

{Producer -> Channel A, Producer -> Channel B} -> Fan In -> Chnnel C -> Consumer

# Fan Out

Producer -> Channel A -> Fan Out -> {Channel B -> Consumer, Channel C -> Consumer}

In [1]:
import (
    "fmt"
    "math/rand"
    "time"
)

In [7]:
func sleep() {
    time.Sleep(time.Duration(rand.Intn(3000)) * time.Millisecond)
}

func producer(ch chan <- int, name string) {
    for {
        //sleep some random time
        sleep()
        
        n := rand.Intn(100)
        
        fmt.Printf("Channel %s -> %d\n", name, n)
        ch <- n
    }
}

func consumer(ch <- chan int) {
    for n := range ch {
        fmt.Printf("<- %d\n", n)
    }
}

func fanIn(chA, chB <- chan int, chC chan <- int) {
    var n int
    for {
        select {
        case n = <- chA:
            chC <- n
        case n = <- chB:
            chC <- n
        }
    }
}

In [8]:
chA := make(chan int)
chB := make(chan int)
chC := make(chan int)

go producer(chA, "A")
go producer(chB, "B")
go consumer(chC)

fanIn(chA, chB, chC)

Channel B -> 23
<- 23
Channel A -> 2
<- 2
Channel A -> 24
<- 24
Channel B -> 78
<- 78
Channel A -> 86
<- 86
Channel A -> 6
<- 6
Channel B -> 74
<- 74
Channel A -> 99
<- 99
Channel B -> 90
<- 90
Channel A -> 18
<- 18
Channel B -> 90
<- 90
Channel B -> 78
<- 78
Channel A -> 51
<- 51
Channel A -> 53
<- 53


main routine canceled, 2 goroutines canceled, 1 goroutine is hanging


# Fan Out

In [10]:
func sleep() {
    time.Sleep(time.Duration(rand.Intn(3000)) * time.Millisecond)
}

func producer(ch chan <- int) {
    for {
        //sleep some random time
        sleep()
        
        n := rand.Intn(100)
        
        fmt.Printf(" -> %d\n", n)
        ch <- n
    }
}

func consumer(ch <- chan int, name string) {
    for n := range ch {
        fmt.Printf("Consumer %s <- %d\n", name, n)
    }
}

func fanOut(chA <- chan int, chB, chC chan <- int) {
    for n := range chA {
        if n < 50 {
            chB <- n
        } else {
            chC <- n
        }
    }
}

In [11]:
chA := make(chan int)
chB := make(chan int)
chC := make(chan int)

go producer(chA)
go consumer(chB, "B")
go consumer(chC, "C")

fanOut(chA, chB, chC)

 -> 37
Consumer B <- 37
 -> 62
Consumer C <- 62
 -> 38
Consumer B <- 38
 -> 30
Consumer B <- 30
 -> 68
Consumer C <- 68
 -> 6
Consumer B <- 6
 -> 18
Consumer B <- 18
 -> 83
Consumer C <- 83
 -> 40
Consumer B <- 40
 -> 27
Consumer B <- 27
 -> 0
Consumer B <- 0
 -> 80
Consumer C <- 80
 -> 70
Consumer C <- 70


main routine is hanging, 3 goroutines are hanging
